# Как взвесить осла?

Ослы играют важную роль в сельской местности Кении. Они нужны людям для перевозки урожая и воды, вспашки полей и для личного транспорта. Когда осел заболевает, ветеринару необходимо выяснить, сколько весит осел, чтобы назначить нужное количество лекарства.

Но у многих ветеринаров в сельской местности Кении нет доступа к весам, чтобы взвесить осла, поэтому им нужно угадать, сколько весит осел.

Слишком мало лекарств и инфекция может повториться, слишком много лекарств - можно вызвать опасную передозировку.

В Кении насчитывается более 1,8 миллиона ослов, поэтому важно иметь простой и точный способ оценки веса осла.

Поля датасета:
1. BCS - оценка состояния тела: от 1 (истощенный) через 3 (нормальный) до 5 (страдающий ожирением) с шагом 0.5;
2. Age - возраст в годах, младше 2, 2-5, 5-10, 10-15, 15-20, и старше 20 лет;
4. Sex - пол;
5. Length - длинна тела (см);
6. Girth - обхват (по шее в см);
7. Height - высота в холке (см);
8. Weight - вес осла (целевая переменная).

Обхват измеряется вокруг туловища сразу за передними ногами, высота измеряется от земли до того места, где шея соединяется с верхней частью спины, а длина измеряется от переднего локтевого сустава до задней части таза.

# Область работы 1 (библиотеки)

In [ ]:
# Данный блок в области 1 НЕ выполняется преподавателем
#
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
# запрещается скрывать предупреждения системы
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
# pip install

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint
import pandas as pd
from IPython.display import display, Image
import seaborn as sns
from scipy.stats import gaussian_kde

from sklearn.compose import ColumnTransformer , make_column_transformer, TransformedTargetRegressor
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.model_selection import cross_val_score, KFold, StratifiedShuffleSplit, GridSearchCV
from sklearn.pipeline import Pipeline,make_pipeline

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, Normalizer,PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder,QuantileTransformer,PowerTransformer
from sklearn.datasets import make_classification
from sklearn.metrics import precision_score, classification_report, confusion_matrix, ConfusionMatrixDisplay,log_loss
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,mean_absolute_percentage_error,f1_score,accuracy_score
from sklearn.metrics import recall_score, zero_one_loss, roc_auc_score, balanced_accuracy_score
import category_encoders as ce
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score, mean_absolute_percentage_error

from category_encoders import TargetEncoder
from sklearn.linear_model import ElasticNet, Ridge, Lasso, Lars,SGDRegressor,LogisticRegression, LinearRegression
from sklearn.svm import SVR,SVC

from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingRegressor,VotingRegressor
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.ensemble import VotingClassifier, StackingClassifier

from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer , make_column_transformer, TransformedTargetRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.covariance import EllipticEnvelope
from sklearn.svm import OneClassSVM
from sklearn import svm
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler,LabelEncoder,FunctionTransformer

from sklearn.neighbors import LocalOutlierFactor
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest, f_classif

import pickle

import time
import re as re

from matplotlib.colors import ListedColormap
from collections import OrderedDict

# pip install category-encoders

from sklearn import tree

import sklearn
print(sklearn.__version__)


1.2.2


# Область работы 2 (финальная модель)

In [ ]:
# Данный блок(и) в области 2 выполняется преподавателем
#
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными,
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать,
# тюниговать и т.д. модель машинного обучения

In [4]:
# Путь к тренировочному набору
 # содержит только имя файла, без имен папок
# Путь к тестовому набору
df = pd.read_csv("train.csv")
df.head() # содержит только имя файла, без имен папок

,BCS,Age,Sex,Length,Girth,Height,Weight
0,2.5,10-15,gelding,89,121,106,158
1,2.5,10-15,stallion,98,117,105,145
2,2.5,2-5,stallion,89,107,100,109
3,2.5,10-15,female,98,120,103,170
4,2.5,2-5,female,87,101,93,110


In [5]:
cat_features = ['Sex', 'Age']
num_features = ['BCS', 'Height', 'Girth', 'Length']

In [6]:
X = df.drop(columns = ['Weight'], axis=1)
y = df['Weight']

In [7]:
transformer_cat = Pipeline(steps=[
    ('onehot', OneHotEncoder(sparse_output = False, handle_unknown='ignore').set_output(transform='pandas')),
    ('scaler', StandardScaler())
])

transformer_num = Pipeline(steps=[
    ('scaler', StandardScaler())
])

CT = ColumnTransformer([
        ("num", transformer_num, num_features),
        ("cat", transformer_cat, cat_features),
        ])

display(CT)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['BCS', 'Height', 'Girth', 'Length']),
                                ('cat',
                                 Pipeline(steps=[('onehot',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False)),
                                                 ('scaler', StandardScaler())]),
                                 ['Sex', 'Age'])])

In [9]:
gb = Pipeline([('preprocessing', CT), ('regressor', GradientBoostingRegressor(
                                    learning_rate=0.01,
                                subsample=0.1,
                                min_samples_split = 2,
                                max_depth = 3,
                                  n_estimators=1000,
                                random_state=52))])


gb.fit(X, y)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['BCS', 'Height', 'Girth',
                                                   'Length']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Sex', 'Age'])])),
                ('regressor',
                 GradientBoostingRegressor(learning_rate=0.01,
                                           n_estimators=1000, random_state=52,
                                           subsample=0.1))])

In [10]:
test = pd.read_csv('test.csv')
test

,BCS,Age,Sex,Length,Girth,Height
0,2.5,5-10,female,93,114,97
1,3.0,5-10,stallion,88,122,102
2,2.5,<2,stallion,80,98,97
3,2.5,<2,stallion,79,100,90
4,2.5,<2,female,85,108,93
...,...,...,...,...,...,...
159,3.0,2-5,stallion,92,112,98
160,2.5,10-15,female,96,116,100
161,3.0,10-15,female,101,123,108
162,2.5,10-15,stallion,98,115,104


In [12]:
y_pred_best = gb.predict(test)

In [13]:
y_pred_best.shape

(164,)